### Structured Output Parser

In [9]:
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.chains import LLMChain

# Load API key
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)

schemas = [
    ResponseSchema(name="language",description="Detected language of the text"),
    ResponseSchema(name="sentiment", description="Sentiment of the text (positive/negative/neutral)")
]

output_parser = StructuredOutputParser.from_response_schemas(schemas)

format_instructions = output_parser.get_format_instructions()
template = """
Analyze the following text:
{text}

{format_instructions}
"""
prompt = PromptTemplate(template=template, 
                        input_variables=["text"], 
                        partial_variables={"format_instructions": format_instructions})

chain = LLMChain(llm=llm, prompt=prompt)
#response = chain.run(text="I love learning about AI, it makes me very excited.")
response = chain.run(text="எனக்கு AI பற்றி கற்றுக்கொள்வது மிகவும் பிடிக்கும், அது என்னை மிகவும் உற்சாகப்படுத்துகிறது.")

parsed = output_parser.parse(response)

print(parsed)

{'language': 'Tamil', 'sentiment': 'positive'}


### pydantic output parser

In [11]:
from datetime import datetime
from pydantic import BaseModel
from langchain_core.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# Define schema
class EventDate(BaseModel):
    event: str
    date: datetime

parser = PydanticOutputParser(pydantic_object=EventDate)

prompt = PromptTemplate(template="""Extract the event name and date.Return in the format:{format_instructions}Text:{input_text}""",
    input_variables=["input_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

chain = prompt | llm | parser

result = chain.invoke({
    "input_text": "The conference will happen on 25 December 2025 in Delhi."
})

print(result)
print(type(result.date))


event='conference' date=datetime.datetime(2025, 12, 25, 0, 0)
<class 'datetime.datetime'>


### Json Output Parser

In [15]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

parser = JsonOutputParser()

prompt = PromptTemplate.from_template(
    "Provide a JSON with keys: name, role."
)

chain = prompt | llm | parser

print(chain.invoke({}))


{'name': 'Alice Smith', 'role': 'Software Engineer'}
